# Model de prediction de diabète

In [1]:
# importation de librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder,LabelBinarizer,MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score,accuracy_score,f1_score
import joblib
import os

In [2]:
os.getcwd()

'D:\\Projets\\Diabete_Regression'

In [4]:
# importation du dataset
df = pd.read_csv(r"D:\Projets\Diabete_Regression\diabete_prediction_dataset.csv")
df

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,0,0,No Info,27.32,6.2,90,0
99996,Female,2.0,0,0,No Info,17.37,6.5,100,0
99997,Male,66.0,0,0,former,27.83,5.7,155,0
99998,Female,24.0,0,0,never,35.42,4.0,100,0


In [5]:
df.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [6]:
df.tail()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
99995,Female,80.0,0,0,No Info,27.32,6.2,90,0
99996,Female,2.0,0,0,No Info,17.37,6.5,100,0
99997,Male,66.0,0,0,former,27.83,5.7,155,0
99998,Female,24.0,0,0,never,35.42,4.0,100,0
99999,Female,57.0,0,0,current,22.43,6.6,90,0


In [7]:
df.shape

(100000, 9)

In [8]:
df.columns

Index(['gender', 'age', 'hypertension', 'heart_disease', 'smoking_history',
       'bmi', 'HbA1c_level', 'blood_glucose_level', 'diabetes'],
      dtype='object')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


In [10]:
df.isna().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64

In [11]:
df.duplicated().sum()

np.int64(3854)

In [12]:
df = df.drop_duplicates()
df

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
99994,Female,36.0,0,0,No Info,24.60,4.8,145,0
99996,Female,2.0,0,0,No Info,17.37,6.5,100,0
99997,Male,66.0,0,0,former,27.83,5.7,155,0
99998,Female,24.0,0,0,never,35.42,4.0,100,0


In [13]:
df["diabetes"].value_counts()

diabetes
0    87664
1     8482
Name: count, dtype: int64

In [14]:
#df["gender"].replace(["Male","Female"],[0,1],inplace=True)


In [15]:
transformation = ColumnTransformer([('cat',OneHotEncoder(),["gender","smoking_history"])],remainder="passthrough")

In [16]:
x = df.drop(["diabetes"],axis=1)
y = df["diabetes"]

In [17]:
#model 

In [18]:
x_trans = transformation.fit_transform(x)
x_trans

array([[  1.  ,   0.  ,   0.  , ...,  25.19,   6.6 , 140.  ],
       [  1.  ,   0.  ,   0.  , ...,  27.32,   6.6 ,  80.  ],
       [  0.  ,   1.  ,   0.  , ...,  27.32,   5.7 , 158.  ],
       ...,
       [  0.  ,   1.  ,   0.  , ...,  27.83,   5.7 , 155.  ],
       [  1.  ,   0.  ,   0.  , ...,  35.42,   4.  , 100.  ],
       [  1.  ,   0.  ,   0.  , ...,  22.43,   6.6 ,  90.  ]],
      shape=(96146, 15))

In [19]:
normalisation = MinMaxScaler()
normalisation.fit(x_trans)

,feature_range,"(0, ...)"
,copy,True
,clip,False


In [20]:
x_train,x_test,y_train,y_test = train_test_split(x_trans,y,test_size=0.2,random_state=42)

In [21]:
model = LogisticRegression(max_iter=100,random_state=42)
model.fit(x_train,y_train)

C:\Users\Destin Biringanine\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [22]:
y_pred = model.predict(x_test)
y_pred

array([0, 0, 1, ..., 0, 0, 0], shape=(19230,))

In [23]:
model.score(x_train,y_train)

0.9580191377606739

In [24]:
accuracy_score(y_test, y_pred)

0.9560582423296932

In [25]:
f1_score(y_test,y_pred)

0.712682760965658

In [26]:
joblib.dump(model,"model_plk")

['model_plk']